In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff
import os
from deepcell.applications import Mesmer
from deepcell.utils.plot_utils import make_outline_overlay
from deepcell.utils.tracking_utils import trks_stats, load_trks
from deepcell.utils.plot_utils import create_rgb_image
from deepcell.utils.transform_utils import pixelwise_transform
import skimage.io as io
import skimage.segmentation

In [16]:
def Mesmer_segmentor(in_dir, save_dir):
    '''
    Produces binary masks for evaluation using Mesmer segementation algorithm

    Parameters
    in_dir: str
        the path to the directory containing the grayscaled DAPI channel images of various fields
    save_dir: str
        the path to the directory where the binary masks will be saved. Each tiff image in the input directory will
        have a corresponding binary mask in the save directory
    '''
    
    # iterating over all files in the input directory
    for file in os.listdir(in_dir):
        # only processes tiff images
        if file[-5:]=='.tiff' or file[-4:]=='.tif':
            # reads the grayscaled DAPI channel of each field as an array
            DAPI_gray = np.array([io.imread(os.path.join(in_dir, file))])
            # first and third channels should be zero for Mesmer to perform nuclear segmentation
            DAPI_gray[:, :, : , 0] = 0
            DAPI_gray[:, :, : , 2] = 0
            # Creates an "rgb image" where green channel is nucleus and the blue channel is cytoplasm (zeros over
            # here since performing only nuclear segmentation)
            rgb_images = create_rgb_image(DAPI_gray[:, :, :, 1:], channel_colors=['green', 'blue'])
            # predicting nuclei using the Mesmer model
            app = Mesmer()
            segmentation_predictions = app.predict(rgb_images[:, :, :, 1:], image_mpp=0.5, compartment='nuclear')
            # extracting nuclei label array
            label_mask = segmentation_predictions[0, :, :, 0]
            # Post-processing of prediction label arrays into binary masks for evaluation. The overall effect of
            # step is that nuclei that are sparse are unchanged but nuclei that are touching are eroded by one
            # pixel where they touch other nuclei. This is necessary for binary connected component analysis during
            # F1-score calculation for segmentation evaluation.
            # Finding the pixels that are touching between any two nuclei
            boundary_bool = skimage.segmentation.find_boundaries(label_mask, connectivity=label_mask.ndim,
                                                                 mode='outer', background=0)
            # Converting these pixels to the background value in the label array
            label_mask[boundary_bool] = 0
            # Converting the label array into a binary mask of foreground (255) and background (0)
            nuclei_mask_final = np.zeros((label_mask.shape[0], label_mask.shape[1]))
            nuclei_mask_final[label_mask != 0] = 255
            nuclei_mask_final = np.uint8(nuclei_mask_final)
            # saving the binary mask in the save directory
            io.imsave(os.path.join(save_dir, file[:file.find('.')] + '.tiff'), nuclei_mask_final)
        else:
            pass
        
    return None

In [19]:
# Replace the strings for in_dir and save_dir to paths on the local machine
in_dir = r'C:\Users\mrl\Desktop\nuclear_seg\DAPI_grayscale'
save_dir = r'C:\Users\mrl\Desktop\nuclear_seg\Mesmer_Binary_masks'
# To create binary masks using Mesmer segmentation algorithm
Mesmer_segmentor(in_dir, save_dir)